***Setup Environment & Load Libraries***

In [ ]:
# Install required libraries
!pip install torch torchvision numpy matplotlib scikit-learn tqdm

# Import necessary libraries
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt

***Load & Preprocess Dataset***

In [ ]:
import os
os.listdir("/content/")

['.config', 'TQVCD-main.zip', 'TQVCD', 'sample_data']

In [ ]:
import zipfile
import os

zip_path = "/content/TQVCD-main.zip"  # Ensure this matches the actual filename
extract_path = "/content/TQVCD"

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted successfully!")

✅ Dataset extracted successfully!


In [ ]:
total_images = sum([len(files) for _, _, files in os.walk("/content/TQVCD")])
print(f"✅ Total images in dataset: {total_images}")

✅ Total images in dataset: 268


***Organize Dataset (Binary Classification)***

In [ ]:
import shutil

# Create binary classification folders
os.makedirs("/content/TQVCD_Binary/normal", exist_ok=True)
os.makedirs("/content/TQVCD_Binary/damaged", exist_ok=True)

# Define dataset folder mappings
source_folders = {
    "normal": ["FN", "RN"],  # Undamaged cars
    "damaged": ["FB", "FC", "RB", "RC"]  # Damaged cars
}

# Move images into binary folders
for label, folders in source_folders.items():
    for folder in folders:
        folder_path = os.path.join("/content/TQVCD", folder)
        if os.path.exists(folder_path):
            for img_name in os.listdir(folder_path):
                img_src = os.path.join(folder_path, img_name)
                img_dst = os.path.join(f"/content/TQVCD_Binary/{label}", img_name)
                shutil.move(img_src, img_dst)

print("✅ Dataset reorganized into binary classification format!")

✅ Dataset reorganized into binary classification format!


***Load and Preprocess Dataset***

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split

# Define dataset path (Binary classification)
DATA_DIR_BINARY = "/content/TQVCD_Binary"

# Data augmentation for training
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomAffine(15, translate=(0.2, 0.2), shear=10),  # Stronger transformations
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Simple transform for validation and testing
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
dataset = datasets.ImageFolder(DATA_DIR_BINARY, transform=transform_train)

# Split dataset (60% Train, 20% Validation, 20% Test)
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Apply different transformations for validation and test sets
val_dataset.dataset.transform = transform_test
test_dataset.dataset.transform = transform_test

# Define DataLoaders
BATCH_SIZE = 16  # Small dataset → Small batch size
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Print dataset split
print(f"✅ Dataset loaded successfully!")
print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")

✅ Dataset loaded successfully!
Train samples: 72, Validation samples: 24, Test samples: 24


***Define ResNet-50 Model (Transfer Learning)***

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained ResNet-50
model = models.resnet50(pretrained=True)

# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False

# Modify final layer for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # Single output neuron for binary classification

model = model.to(device)

# Define Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, weight_decay=5e-4)

print("✅ ResNet-50 Model Loaded & Ready!")

Using device: cpu


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 123MB/s]


✅ ResNet-50 Model Loaded & Ready!


***Implement Grid Search for Hyperparameter Tuning***

In [ ]:
from itertools import product
import time

# Define reduced hyperparameter grid
LEARNING_RATES = [1e-3, 1e-4]  # Focus on stable training
BATCH_SIZES = [8, 16]  # Small dataset → Small batch sizes
EPOCHS_LIST = [20, 30]  # Testing different training durations
OPTIMIZERS = ["adam", "sgd"]  # Adam adapts, SGD is robust

best_model_params = None
best_val_loss = float("inf")

# Iterate over all hyperparameter combinations
for lr, bs, ep, opt in product(LEARNING_RATES, BATCH_SIZES, EPOCHS_LIST, OPTIMIZERS):
    print(f"\n🔍 Testing: LR={lr}, BS={bs}, Epochs={ep}, Optimizer={opt}")

    # Define DataLoaders with current batch size
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False)

    # Load pre-trained ResNet-50 model
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)  # Binary classification
    model = model.to(device)

    # Define Loss Function
    criterion = nn.BCEWithLogitsLoss()

    # Define Optimizer
    if opt == "adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif opt == "sgd":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Training Loop with Early Stopping
    patience = 5
    best_loss = float("inf")
    early_stop_count = 0

    for epoch in range(ep):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device, dtype=torch.float32)

            optimizer.zero_grad()
            outputs = model(images).squeeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device, dtype=torch.float32)
                outputs = model(images).squeeze(1)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch [{epoch+1}/{ep}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Early Stopping
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            early_stop_count = 0
        else:
            early_stop_count += 1
            if early_stop_count >= patience:
                print("⏹ Early stopping triggered.")
                break

    # Track Best Hyperparameter Set
    if best_loss < best_val_loss:
        best_val_loss = best_loss
        best_model_params = (lr, bs, ep, opt)

print(f"\n🔥 Best Hyperparameters: LR={best_model_params[0]}, BS={best_model_params[1]}, Epochs={best_model_params[2]}, Optimizer={best_model_params[3]}")


🔍 Testing: LR=0.001, BS=8, Epochs=20, Optimizer=adam
Epoch [1/20] - Train Loss: 0.8692, Val Loss: 31.8546
Epoch [2/20] - Train Loss: 0.7328, Val Loss: 1.1490
Epoch [3/20] - Train Loss: 0.4738, Val Loss: 1.9574
Epoch [4/20] - Train Loss: 0.3306, Val Loss: 0.7494
Epoch [5/20] - Train Loss: 0.3155, Val Loss: 1.4959
Epoch [6/20] - Train Loss: 0.3561, Val Loss: 2.2659
Epoch [7/20] - Train Loss: 0.4033, Val Loss: 0.8900
Epoch [8/20] - Train Loss: 0.2269, Val Loss: 0.9737
Epoch [9/20] - Train Loss: 0.2490, Val Loss: 0.7823
⏹ Early stopping triggered.

🔍 Testing: LR=0.001, BS=8, Epochs=20, Optimizer=sgd
Epoch [1/20] - Train Loss: 0.6989, Val Loss: 0.8045
Epoch [2/20] - Train Loss: 0.5840, Val Loss: 0.6765
Epoch [3/20] - Train Loss: 0.5186, Val Loss: 0.6817
Epoch [4/20] - Train Loss: 0.4436, Val Loss: 0.6225
Epoch [5/20] - Train Loss: 0.3424, Val Loss: 0.6321
Epoch [6/20] - Train Loss: 0.2459, Val Loss: 0.5914
Epoch [7/20] - Train Loss: 0.1663, Val Loss: 0.6645
Epoch [8/20] - Train Loss: 0.105

***Train Final Model with Best Hyperparameters***

In [ ]:
# Extract Best Hyperparameters
BEST_LR, BEST_BS, BEST_EPOCHS, BEST_OPTIMIZER = best_model_params

# Update DataLoader with Best Batch Size
train_loader = DataLoader(train_dataset, batch_size=BEST_BS, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BEST_BS, shuffle=False)

# Load Pre-trained ResNet-50 Model
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # Binary classification
model = model.to(device)

# Define Loss & Optimizer
criterion = nn.BCEWithLogitsLoss()
if BEST_OPTIMIZER == "adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=BEST_LR)
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=BEST_LR, momentum=0.9)

# Train Model with Best Parameters
best_model_loss = float("inf")
for epoch in range(BEST_EPOCHS):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device, dtype=torch.float32)
            outputs = model(images).squeeze(1)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{BEST_EPOCHS}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    # Save Best Model
    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        torch.save(model.state_dict(), "best_tuned_model.pth")
        print("✅ Best model saved!")


Epoch [1/20] - Train Loss: 0.6454, Val Loss: 0.6070
✅ Best model saved!
Epoch [2/20] - Train Loss: 0.2530, Val Loss: 0.6654
Epoch [3/20] - Train Loss: 0.1233, Val Loss: 0.6281
Epoch [4/20] - Train Loss: 0.0397, Val Loss: 0.6414
Epoch [5/20] - Train Loss: 0.0664, Val Loss: 0.6826
Epoch [6/20] - Train Loss: 0.0452, Val Loss: 0.6404
Epoch [7/20] - Train Loss: 0.1020, Val Loss: 0.7445
Epoch [8/20] - Train Loss: 0.1899, Val Loss: 0.7852
Epoch [9/20] - Train Loss: 0.0535, Val Loss: 0.7457
Epoch [10/20] - Train Loss: 0.0420, Val Loss: 0.8006
Epoch [11/20] - Train Loss: 0.0557, Val Loss: 0.9099
Epoch [12/20] - Train Loss: 0.0072, Val Loss: 0.9178
Epoch [13/20] - Train Loss: 0.0646, Val Loss: 0.9653
Epoch [14/20] - Train Loss: 0.0331, Val Loss: 0.9886
Epoch [15/20] - Train Loss: 0.0754, Val Loss: 0.9184
Epoch [16/20] - Train Loss: 0.0241, Val Loss: 1.1250
Epoch [17/20] - Train Loss: 0.0170, Val Loss: 1.0800
Epoch [18/20] - Train Loss: 0.0773, Val Loss: 1.4711
Epoch [19/20] - Train Loss: 0.0514,

***Evaluate Final Model***

In [ ]:
# Load Best Model
model.load_state_dict(torch.load("best_tuned_model.pth"))
model.eval()

y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.cpu().numpy()

        outputs = model(images).squeeze(1)
        preds = torch.sigmoid(outputs).cpu().numpy() > 0.5  # Convert logits to binary values (0 or 1)

        y_true.extend(labels)
        y_pred.extend(preds)

# Compute Metrics
print(f"\n🔹 Final Test Results:")
print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"Precision: {precision_score(y_true, y_pred):.4f}")
print(f"Recall: {recall_score(y_true, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")

<ipython-input-12-b27e64a25d16>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_tuned_model.pth"))



🔹 Final Test Results:
Accuracy: 0.7917
Precision: 1.0000
Recall: 0.2857
F1 Score: 0.4444
